In [222]:
import psycopg2
import pandas as pd
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from collections import Counter
from nltk.corpus import stopwords
import string
import numpy as np
from gensim.models import Phrases
from gensim.models.word2vec import LineSentence

In [223]:
def retrieve_reviews(product):
    conn = psycopg2.connect(dbname='product_reviews', user='postgres', password='', host='localhost')
    cursor = conn.cursor()
    p_no = 0
    r_no = 0
    cursor.execute("select * from reviews where product_name = '{}'".format(product))
    rows = cursor.fetchall()
    df = pd.DataFrame(rows)
    cursor.close()
    conn.close()
    return df

In [224]:
df = retrieve_reviews('Samsung Electronics UN65MU6300 65-Inch 4K Ultra HD Smart LED TV (2017 Model)')

In [225]:
df.head(2)

,0,1,2,3,4,5,6,7,8,9,10
0,475,Samsung Electronics UN65MU6300 65-Inch 4K Ultr...,https://www.amazon.com/Samsung-Electronics-UN6...,1,1,5.0 out of 5 stars,"August 28, 2017",James Barrow,Someone here complained that the television is...,"Before I purchased this television, I read alm...",Verified Purchase Helpful No...
1,476,Samsung Electronics UN65MU6300 65-Inch 4K Ultr...,https://www.amazon.com/Samsung-Electronics-UN6...,1,2,5.0 out of 5 stars,"June 18, 2017",Raul G.,"Incredible Picture Quality, Excellent Features...",Its been about seven years since I purchased m...,TOP 100 REVIEWERVINE VOICESize: 50-Inch|Style: TV


In [226]:
columns = ['id','p_name','url','p_no','r_no','r_stars','r_date','r_name','r_title','r_text','r_comments']
df.columns = columns
df.head(2)

,id,p_name,url,p_no,r_no,r_stars,r_date,r_name,r_title,r_text,r_comments
0,475,Samsung Electronics UN65MU6300 65-Inch 4K Ultr...,https://www.amazon.com/Samsung-Electronics-UN6...,1,1,5.0 out of 5 stars,"August 28, 2017",James Barrow,Someone here complained that the television is...,"Before I purchased this television, I read alm...",Verified Purchase Helpful No...
1,476,Samsung Electronics UN65MU6300 65-Inch 4K Ultr...,https://www.amazon.com/Samsung-Electronics-UN6...,1,2,5.0 out of 5 stars,"June 18, 2017",Raul G.,"Incredible Picture Quality, Excellent Features...",Its been about seven years since I purchased m...,TOP 100 REVIEWERVINE VOICESize: 50-Inch|Style: TV


In [261]:
df_new = df.copy()
df_new = df_new[df_new['r_stars'].apply(lambda x: x in ['5.0 out of 5 stars','1.0 out of 5 stars'])]
df_new['rating'] = df_new['r_stars'].apply(lambda x: 1 if x == '5.0 out of 5 stars' else 0)
df_new = df_new[['rating','r_name','r_text']]
df_new.head(3)

,rating,r_name,r_text
0,1,James Barrow,"Before I purchased this television, I read alm..."
1,1,Raul G.,Its been about seven years since I purchased m...
2,1,Y. L.,WATCH OUT FOR the TV ADS AGREEMENT DURING SET ...


In [262]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/bhaskarballapragada/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/bhaskarballapragada/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [263]:
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
stopwords = set(stopwords.words('english'))
punctuation = set(string.punctuation)
lemmatize = WordNetLemmatizer()

def cleaning(article):
    one = " ".join([i for i in article.lower().split() if i not in stopwords])
    two = "".join(i for i in one if i not in punctuation)
    three = " ".join(lemmatize.lemmatize(i) for i in two.split())
    return three


In [264]:
df_text = df_new[['r_text']]

In [265]:
df_text.info()

data = df_text.applymap(cleaning)['r_text']
text = data
text_list = [i.split() for i in text]
len(text_list)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1259 entries, 0 to 1666
Data columns (total 1 columns):
r_text    1259 non-null object
dtypes: object(1)
memory usage: 19.7+ KB


In [272]:
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
 
NUM_TOPICS = 10
 
vectorizer = CountVectorizer(min_df=5, max_df=0.9, 
                             stop_words='english', lowercase=True, 
                             token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')
data_vectorized = vectorizer.fit_transform(data)
 
# Build a Latent Dirichlet Allocation Model
lda_model = LatentDirichletAllocation(n_topics=NUM_TOPICS, max_iter=10, learning_method='online')
lda_Z = lda_model.fit_transform(data_vectorized)
print(lda_Z.shape)  # (NO_DOCUMENTS, NO_TOPICS)
 
# Build a Non-Negative Matrix Factorization Model
nmf_model = NMF(n_components=NUM_TOPICS)
nmf_Z = nmf_model.fit_transform(data_vectorized)
print(nmf_Z.shape)  # (NO_DOCUMENTS, NO_TOPICS)
 
# Build a Latent Semantic Indexing Model
lsi_model = TruncatedSVD(n_components=NUM_TOPICS)
lsi_Z = lsi_model.fit_transform(data_vectorized)
print(lsi_Z.shape)  # (NO_DOCUMENTS, NO_TOPICS)
 
 
# Let's see how the first document in the corpus looks like in different topic spaces
print(lda_Z[0])
print(nmf_Z[0])
print(lsi_Z[0])
 

(1259, 10)
(1259, 10)
(1259, 10)
[0.0008067  0.33955228 0.56705054 0.00080656 0.00080662 0.00080657
 0.00080657 0.00080662 0.08775084 0.0008067 ]
[0.30178955 0.         0.04811616 0.07516771 0.         0.
 0.         2.82397108 0.         0.41960025]
[ 6.92725336 -0.58631147  0.80229903  7.64852866  5.09987315  4.2246399
 -2.76300351 -3.22398303  3.07185207 -3.60388134]


In [274]:
def print_topics(model, vectorizer, top_n=10):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-top_n - 1:-1]])
 
print("LDA Model:")
print_topics(lda_model, vectorizer)
print("=" * 20)
 
print("NMF Model:")
print_topics(nmf_model, vectorizer)
print("=" * 20)
 
print("LSI Model:")
print_topics(lsi_model, vectorizer)
print("=" * 20)

LDA Model:
Topic 0:
[('smart', 55.89289787559572), ('netflix', 36.87943025311114), ('apps', 33.03166206435023), ('app', 24.97157981114575), ('fine', 19.77851499132135), ('samsung', 18.779291393221104), ('work', 17.881018579681957), ('great', 15.327118194323555), ('hulu', 15.077669830124028), ('like', 14.777069299391526)]
Topic 1:
[('picture', 183.39760616518237), ('love', 147.11431934158568), ('great', 108.38880372612324), ('remote', 99.93107515337516), ('sound', 98.96559666474855), ('set', 78.82782673625032), ('samsung', 72.78984585956619), ('color', 70.28026226265486), ('quality', 66.16183189869399), ('good', 63.30537208597287)]
Topic 2:
[('samsung', 166.55220137703037), ('television', 81.52973662933776), ('screen', 80.21981191568182), ('problem', 78.99268100117665), ('time', 65.19181088225282), ('year', 58.42212719787783), ('product', 56.861985416612484), ('ive', 53.71351795700617), ('old', 49.026331118836545), ('black', 46.40903271128121)]
Topic 3:
[('space', 4.914961870730656), ('

In [276]:
import pandas as pd
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, LabelSet
output_notebook()

Loading BokehJS ...

In [277]:

svd = TruncatedSVD(n_components=2)
documents_2d = svd.fit_transform(data_vectorized)
 
df = pd.DataFrame(columns=['x', 'y', 'document'])
df['x'], df['y'], df['document'] = documents_2d[:,0], documents_2d[:,1], range(len(data))
 
source = ColumnDataSource(ColumnDataSource.from_df(df))
labels = LabelSet(x="x", y="y", text="document", y_offset=8,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
 
plot = figure(plot_width=600, plot_height=600)
plot.circle("x", "y", size=12, source=source, line_color="black", fill_alpha=0.8)
plot.add_layout(labels)
show(plot, notebook_handle=True)

In [278]:

svd = TruncatedSVD(n_components=2)
words_2d = svd.fit_transform(data_vectorized.T)
 
df = pd.DataFrame(columns=['x', 'y', 'word'])
df['x'], df['y'], df['word'] = words_2d[:,0], words_2d[:,1], vectorizer.get_feature_names()
 
source = ColumnDataSource(ColumnDataSource.from_df(df))
labels = LabelSet(x="x", y="y", text="word", y_offset=8,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
 
plot = figure(plot_width=600, plot_height=600)
plot.circle("x", "y", size=12, source=source, line_color="black", fill_alpha=0.8)
plot.add_layout(labels)
show(plot, notebook_handle=True)

In [279]:

from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
 
NUM_TOPICS = 10
 
vectorizer = CountVectorizer(min_df=5, max_df=0.9, 
                             stop_words='english', lowercase=True, 
                             token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')
data_vectorized = vectorizer.fit_transform(data)
 
# Build a Latent Dirichlet Allocation Model
lda_model = LatentDirichletAllocation(n_topics=NUM_TOPICS, max_iter=10, learning_method='online')
lda_Z = lda_model.fit_transform(data_vectorized)
 
text = "The economy is working better than ever"
x = lda_model.transform(vectorizer.transform([text]))[0]
print(x, x.sum())

[0.03333333 0.3169332  0.03333333 0.03333668 0.03333616 0.03333686
 0.03334312 0.03333333 0.03334474 0.41636925] 1.0


In [286]:
import pyLDAvis.sklearn
 
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer, mds='tsne')
panel

/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


PreparedData(topic_coordinates=            Freq  cluster  topics           x           y
topic                                                    
1      19.354007        1       1 -268.869781 -132.553223
8      18.431954        1       2  246.103134  126.564903
5      16.840385        1       3  156.734848 -112.946297
6      14.656229        1       4 -191.697083  312.209625
9       9.724021        1       5 -335.967560   88.068161
3       9.051290        1       6  -34.677776 -260.249390
7       5.986281        1       7   44.270481  290.670929
4       4.250631        1       8   54.706669   78.794296
0       0.908134        1       9  -76.203018  -65.570244
2       0.797067        1      10 -135.857132  119.967216, topic_info=     Category        Freq        Term       Total  loglift  logprob
term                                                               
323   Default  490.000000       great  490.000000  30.0000  30.0000
644   Default  475.000000     samsung  475.000000  29.0000  29.0000
435   Default  199.000000        love  199.000000  28.0000  28.0000
536   Default  531.000000     picture  531.000000  27.0000  27.0000
613   Default  289.000000      remote  289.000000  26.0000  26.0000
82    Default  130.000000         box  130.000000  25.0000  25.0000
654   Default  176.000000      screen  176.000000  24.0000  24.0000
671   Default  115.000000     setting  115.000000  23.0000  23.0000
238   Default  185.000000        easy  185.000000  22.0000  22.0000
669   Default   76.000000     service   76.000000  21.0000  21.0000
569   Default   99.000000     product   99.000000  20.0000  20.0000
467   Default   44.000000     monitor   44.000000  19.0000  19.0000
486   Default   95.000000        nice   95.000000  18.0000  18.0000
1     Default   25.000000  absolutely   25.000000  17.0000  17.0000
811   Default  171.000000         use  171.000000  16.0000  16.0000
583   Default  221.000000     quality  221.000000  15.0000  15.0000
97    Default  109.000000         buy  109.000000  14.0000  14.0000
64    Default   60.000000   beautiful   60.000000  13.0000  13.0000
417   Default   35.000000        line   35.000000  12.0000  12.0000
43    Default   51.000000       audio   51.000000  11.0000  11.0000
743   Default   59.000000     support   59.000000  10.0000  10.0000
670   Default  194.000000         set  194.000000   9.0000   9.0000
81    Default   90.000000      bought   90.000000   8.0000   8.0000
562   Default  154.000000       price  154.000000   7.0000   7.0000
181   Default   50.000000    customer   50.000000   6.0000   6.0000
822   Default   54.000000       video   54.000000   5.0000   5.0000
35    Default   89.000000        apps   89.000000   4.0000   4.0000
804   Default   36.000000      update   36.000000   3.0000   3.0000
202   Default   54.000000    delivery   54.000000   2.0000   2.0000
158   Default   70.000000     control   70.000000   1.0000   1.0000
...       ...         ...         ...         ...      ...      ...
875   Topic10    3.961623       youll    7.123997   4.2452  -3.7380
847   Topic10    4.908489      weight   11.779348   3.9566  -3.5237
239   Topic10    3.109620      effect   11.097405   3.5598  -3.9802
1     Topic10    7.139225  absolutely   25.923601   3.5424  -3.1491
348   Topic10    1.819195      higher    7.175905   3.4597  -4.5163
292   Topic10    2.310965        flat   11.576724   3.2207  -4.2770
565   Topic10    3.752977    probably   19.001122   3.2100  -3.7921
675   Topic10    1.172407        ship    6.296974   3.1510  -4.9556
512   Topic10    1.039316  originally    5.615332   3.1450  -5.0761
773   Topic10    1.152877         tip    7.079178   3.0171  -4.9724
519   Topic10    0.977399     package    6.009656   3.0158  -5.1375
530   Topic10    2.497392   perfectly   17.387153   2.8915  -4.1994
701   Topic10    0.930198        soap    6.564428   2.8780  -5.1870
503   Topic10    0.927834       opera    6.563918   2.8755  -5.1896
582   Topic10    0.890881        push    6.523523   2.8410  -5.23